# 무선으로 JoyStick을 활용해 Jetbot을 제어해보자

- 1) http://html5gamepad.com 사이트를 통해 index 확인

In [3]:
import ipywidgets.widgets as widgets
from IPython.display import display

controller = widgets.Controller(index=0)

display(controller)

Controller()

# 유선으로 Jetbot 제어 Logic -> 무선

- JoyStick 제어 + 카메라 확인
- 카메라!!!

In [143]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

import traitlets

camera = Camera.instance(width=224, height=224)
image = widgets.Image(width=224, height=224, format='jpeg')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## camera와 jetbot을 동시에
- 1) 병렬처리 multiprocessing Lib -> 고질적인 문제점 : 잘 안꺼짐
- 2) Thread

In [6]:
# camera 잠깐 꺼두자
camera_link.unlink()

In [12]:
import time

def count_number(start, end):
    for i in range(s2tart, end + 1):
        print(i, end=' ')
        time.sleep(0.5)
    print()

In [13]:
count_number(1, 10)
count_number(101, 110)

1 2 3 4 5 6 7 8 9 10 
101 102 103 104 105 106 107 108 109 110 


In [26]:
import threading # Thread 라이브러리

thread1 = threading.Thread(target=count_number, args=(1, 100))
thread2 = threading.Thread(target=count_number, args=(101, 200))

thread1.start()
thread2.start()

1 101 2 102 3 103 4 104 5 105 6 106 

In [28]:
stop_thread(thread2)

116 

In [17]:
# 외부에서 Thread 능동적으로 멈추는 함수 정의
import inspect # isclass() 
import ctypes # python에서 C언어 관련 Logic 사용, type확인 함수

# stop_thread(멈출 쓰레드)
# 이 함수를 실행하면 쓰레드가 멈춘다!!

def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

## Thread 사용시 외부에서 Thread를 멈출 수 있는 Logic을 설계하자

### Thread란?
- 한글
- 1) 10장 인쇄하기
- 2) 맞춤법 검사
- 3) 표 만들기
- 4) 사진 넣기
- 동시에 동작하는 작업의 최소 단위 : Thread
    

In [122]:
# ServoMotor를 활용한, DC Motor 활용한
# Pan/Tilt, Left/Right 바퀴 제어

from servoserial import ServoSerial
from jetbot import Robot # custom.py 열어보면 Robot클래스를 활용해서 forward() 만들었음

servo = ServoSerial()
robot = Robot()

serial Open!


In [138]:
robot.left_motor.value = 0.0

In [83]:
global pan, tilt
pan, tilt = 2100, 2700

def pan_left():
    global pan
    pan += 1
    servo.Servo_serial_control(1, pan)
    
def pan_right():
    global pan
    pan -= 1
    servo.Servo_serial_control(1, pan)
    
def tilt_up():
    global tilt
    tilt += 1
    servo.Servo_serial_control(2, tilt)
    
def tilt_down():
    global tilt
    tilt -= 1
    servo.Servo_serial_control(2, tilt)
    
def center():
    global pan, tilt
    pan, tilt = 2100, 2700
    servo.Servo_serial_double_control(1, pan, 2, tilt)

In [85]:
center()

89
b'\xff\xff\xfe\x0e\x83*\x04\x01\x084\x00\n\x02\n\x8c\x00\nY'


# Pan/Tilt 기능과 JoyStick을 매칭

In [144]:
import time

def jetbot_motion():
    pan, tilt = 2100, 2700
    left, right = 0.0, 0.0
    
    while 1:
        if controller.axes[2].value == -1:
            pan += 10
            servo.Servo_serial_control(1, pan)
            time.sleep(0.005)
        elif controller.axes[2].value == 1:
            pan -= 10
            servo.Servo_serial_control(1, pan)
            time.sleep(0.005)
            
        if controller.axes[5].value == -1:
            tilt += 10
            servo.Servo_serial_control(2, tilt)
            time.sleep(0.005)
        elif controller.axes[5].value == 1:
            tilt -= 10
            servo.Servo_serial_control(2, tilt)
            time.sleep(0.005)
            
        if controller.buttons[11].value == True:
            pan, tilt = 2100, 2700
            servo.Servo_serial_double_control(1, pan, 2, tilt)
            time.sleep(0.005)
            
        if controller.axes[1].value == -1:
            left += 0.05
            right += 0.05
            
            if left >= 1.0:
                left = 1.0
            if right >= 1.0:
                right = 1.0
            
            robot.left_motor.value = left
            robot.right_motor.value = right
            
            time.sleep(0.005)
            
        if controller.axes[1].value == 1:
            left -= 0.05
            right -= 0.05
            
            if left <= -1.0:
                left = -1.0
            if right <= -1.0:
                right = -1.0
            
            robot.left_motor.value = left
            robot.right_motor.value = right
            
            time.sleep(0.005)
        
        if controller.buttons[10].value == True:
            left, right = 0.0, 0.0
            robot.left_motor.value = left
            robot.right_motor.value = right       
            time.sleep(0.005)
            

In [147]:
import threading
thread = threading.Thread(target=jetbot_motion)
thread.start()

In [146]:
stop_thread(thread)